## Use case: identify and collect information about repositories catering to the medical research community

Medical researchers are looking for a suitable repository to deposit their data. They require a repository catering to medical research that offers data upload and assigns DOIs to datasets.

Repositories meeting these specifications can be identified via the re3data API. The API also provides the option to retrieve further information about these repositories, such as the name of the repository or a description.

### Step 1: load packages

The package **httr** includes the HTTP method GET, which will be used to request data from the re3data API. Responses from the redata API are returned in XML. **xml2** includes functions for working with XML, for example parsing or extracting content of specific elements. If necessary, install the packages before loading them.

In [1]:
#install.packages("htttr")
#install.packages("xml2")
library(httr)
library(xml2)

### Step 2: define query parameters

Information on individual repositories can be extracted using the re3data ID. Therefore, re3data IDs of repositories with the desired characteristics need to be identified first.

The re3data API allows querying via the endpoint **/api/beta/repositories**. Parameters that can be queried are listed in the [re3data API documentaion](https://www.re3data.org/api/doc). For more information on re3data metadata, including descriptions of available elements and controlled vocabularies, please refer to the documentation of the [re3data Metadata Schema](https://doi.org/10.2312/re3.006) (the API uses version 2.2 of the re3data Metadata Schema).  
The query below returns re3data IDs of repositories meeting the following conditions:

* **"subjects[]" = "205 Medicine"** The repository caters to the subject *Medicine*, notation 205 in the DFG Subject Classification, the subject classification used by re3data.
* **"dataUploads[]"="open"** The repository allows data upload.
* **"pidSystems[]"="DOI"** The repository assigns DOIs.

In [2]:
re3data_query <- list("subjects[]" = "205 Medicine", "dataUploads[]"="open", "pidSystems[]"="DOI")

### Step 3: obtain URLs for further API queries

The query parameters defined in the previous step can then be passed to the re3data API using **GET**.  
The XML response is parsed using **read_XML**. XML elements or attributes can be identified using XPath syntax. The response from the re3data API includes URLs for further queries to the **/api/beta/repository** endpoint. These URLs can be identified with a simple XPath expression. All attributes matching the XPath syntax are identified with **xml_find_all**, and their content is extracted using **xml_text**.

The three functions are nested in the example below.

In [3]:
re3data_request <- GET("https://www.re3data.org/api/beta/repositories?", query = re3data_query) 

URLs <- xml_text(xml_find_all(read_xml(re3data_request), xpath = "//@href"))

### Step 4: define what information about the repositories should be requested

The function **extract_repository_info** defined in the following code block points to and extracts the content of specific XML elements and attributes.This function will be used later to extract the spedified information from responses of the re3data API. Its basic structure is similar to the process of extracting the URLs outlined in step 3 above.  
The XPath expressions defined here will extract the re3data IDs, names, URLs, and descriptions of the repositories. Results are stored in a named list that can be processed later.

Depending on specific use cases, this function can be adapted to extract a different set of elements and attributes. For an overview of the metadata re3data offers, please refer to the documentation of the [re3data Metadata Schema](https://doi.org/10.2312/re3.006) (the API uses version 2.2 of the re3data Metadata Schema).

The function **xml_structure** from the package **xml2** can be very useful for inspecting the structure of XML objects and specifying XPath expressions.
    
Please note that in version 2.2 of the re3data Metadata Schema, the elements mentioned here have occurences of 1 or 0-1, meaning that for each repository, they occur once at most. For information on how to deal with elements that can occur multiple times, please refer to other examples for using the re3data API.

In [4]:
extract_repository_info <- function(repository_metadata_XML) {
  list(
    re3data_ID = xml_text(xml_find_all(repository_metadata_XML, "//r3d:re3data.orgIdentifier")),
    repositoryName = xml_text(xml_find_all(repository_metadata_XML, "//r3d:repositoryName")),
    repositoryUrl = xml_text(xml_find_all(repository_metadata_XML, "//r3d:repositoryURL")),
    description = xml_text(xml_find_all(repository_metadata_XML, "//r3d:description"))
  )
}

### Step 5: create a container for storing results

**repository_info** is a container for storing results of the API query. The dataframe has four columns corresponding to names of the list items defined by **extract_repository_info**.

In [5]:
repository_info <- data.frame(matrix(ncol = 4, nrow = 0))

colnames(repository_info) <- c("re3data_ID", "repositoryName", "repositoryUrl", "description")

### Step 6: gather detailed information about repositories

After preparing the list of URLs, the extracting function and the container for results, these components can be put together. The code block below iterates through the list of URLs using a for-loop. For each repository, data is requested from the re3data API using **GET**. The XML response is parsed with **read_xml** before **extract_repository_info** is called. The results are then appended as a new row to **repository_info**.

In [6]:
for (url in URLs) {
  repository_metadata_request <- GET(url)
  repository_metadata_XML <-read_xml(repository_metadata_request) 
  results_list <- extract_repository_info(repository_metadata_XML)
  repository_info <- rbind(repository_info, results_list)
}

### Results

Results are now stored in **repository_info**. They can be inspected using **head**, visualized or stored locally with **write.csv**.

In [7]:
head(repository_info)

,re3data_ID,repositoryName,repositoryUrl,description
,<chr>,<chr>,<chr>,<chr>
1,r3d100012823,Vivli,https://vivli.org/,"Vivli is a non-profit organization working to advance human health through the insights and discoveries gained by sharing and analyzing data. It is home to an independent global data-sharing and analytics platform which serves all elements of the international research community. The platform includes a data repository, in-depth search engine and cloud-based analytics, and harmonizes governance, policy and processes to make sharing data easier. Vivli acts as a neutral broker between data contributor and data user and the wider data sharing community."
2,r3d100010953,Polar Data Catalogue,https://www.polardata.ca/,"The Polar Data Catalogue is an online database of metadata and data that describes, indexes and provides access to diverse data sets generated by polar researchers. These records cover a wide range of disciplines from natural sciences and policy, to health, social sciences, and more."
3,r3d100010261,National Addiction & HIV Data Archive Program,http://www.icpsr.umich.edu/icpsrweb/NAHDAP/,"NAHDAP acquires, preserves and disseminates data relevant to drug addiction and HIV research. By preserving and making available an easily accessible library of electronic data on drug addiction and HIV infection in the United States, NAHDAP offers scholars the opportunity to conduct secondary analysis on major issues of social and behavioral sciences and public policy"
4,r3d100012815,UNB Libraries Dataverse,https://dataverse.lib.unb.ca/,"UNB Dataverse is repository for research data collected by researchers and organizations primarily affiliated with the University of New Brunswick. The repository allows researchers to deposit, share, analyze, cite, and explore data. Dataverse is an open source application developed by the Institute for Quantitative Social Science (IQSS) at Harvard University."
5,r3d100012074,BindingDB,http://bindingdb.org/bind/index.jsp,"BindingDB is a public, web-accessible knowledgebase of measured binding affinities, focusing chiefly on the interactions of proteins considered to be candidate drug-targets with ligands that are small, drug-like molecules. BindingDB supports medicinal chemistry and drug discovery via literature awareness and development of structure-activity relations (SAR and QSAR); validation of computational chemistry and molecular modeling approaches such as docking, scoring and free energy methods; chemical biology and chemical genomics; and basic studies of the physical chemistry of molecular recognition. BindingDB also includes a small collection of host-guest binding data of interest to chemists studying supramolecular systems. The data collection derives from a variety of measurement techniques, including enzyme inhibition and kinetics, isothermal titration calorimetry, NMR, and radioligand and competition assays. BindingDB includes data extracted from the literature and from US Patents by the BindingDB project, selected PubChem confirmatory BioAssays, and ChEMBL entries for which a well defined protein target (""TARGET_TYPE='PROTEIN'"") is provided."
